In [3]:
import os
from dotenv import load_dotenv

# Set environment variables in the .env file.
load_dotenv()

OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]
OPENAI_API_BASE = os.environ["OPENAI_API_BASE"]
OPENAI_DEPLOYMENT_NAME = os.environ["OPENAI_DEPLOYMENT_NAME"]

azure_credential = DefaultAzureCredential(
    exclude_environment_credential=True,
    exclude_managed_identity_credential=True,
    exclude_powershell_credential=True,
    exclude_visual_studio_code_credential=True,
    exclude_shared_token_cache_credential=True,
    exclude_interactive_browser_credential=True
)

In [4]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Use the current user identity to authenticate with Azure OpenAI, Cognitive Search and Blob Storage (no secrets needed, 
# just use 'az login' locally, and managed identity when deployed on Azure). If you need to use keys, use separate AzureKeyCredential instances with the 
# keys for each service
# If you encounter an error here are some ways to troubleshoot:
# - you can exclude the problematic credential by using a parameter (ex. exclude_shared_token_cache_credential=True)
# - For your credential, grant Cognitive Services User and Cognitive Services OpenAI User roles - EVEN IF YOU HAVE OWNER/ADMIN RIGHTS.
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
  azure_endpoint = OPENAI_API_BASE,
  api_version="2024-02-01",
  azure_ad_token_provider=token_provider
)

response = client.chat.completions.create(
    model=OPENAI_DEPLOYMENT_NAME,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

Yes, several other Azure AI services support the use of customer-managed keys for encryption at rest, as part of Azure's commitment to providing robust security options. Customer-managed keys (CMKs) allow customers to control the encryption keys and to implement and manage their own key lifecycle.

For example, Azure Cognitive Services, which includes services for vision, speech, language, decision, and more, allows customers to use their own keys for data encryption, giving them ownership and control over their encryption keys. This feature is very important for enterprises that have strict compliance and regulatory requirements for data security.

To implement customer-managed keys with Azure services, you would typically use Azure Key Vault, which is a cloud service for securely storing and accessing secrets, keys, and certificates. By integrating Azure Key Vault with other Azure services, you ensure that encryption keys are well-managed and protected according to your organization'